In [ ]:
%matplotlib inline
import string
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
matplotlib.rc('xtick', labelsize=14) 
matplotlib.rc('ytick', labelsize=14)

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
sexist_data = pd.read_csv("/content/drive/MyDrive/twitter_sentiment/ISEP Sexist Data labeling.csv")

In [ ]:
sexist_data

,Sentences,Label
0,"For a woman, that is good",1
1,Just dress sexy and you will sign any contract.,1
2,Are you having period ?,1
3,Stop being bitchy,1
4,I leave early because of her children instead ...,1
...,...,...
1132,Opportunities are like night owls. They like t...,0
1133,It's not just a question of doing what you lov...,0
1134,A genius does what he masters. An ordinary man...,0
1135,If you want to know the real reasons behind th...,0


**Data Cleaning**

In [ ]:
sexist_data.dropna()
sexist_data.drop_duplicates()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation
...,...,...,...
31957,31958,0,ate @user isz that youuu?ðððððð...
31958,31959,0,to see nina turner on the airwaves trying to...
31959,31960,0,listening to sad songs on a monday morning otw...
31960,31961,1,"@user #sikh #temple vandalised in in #calgary,..."


In [ ]:
from gensim.parsing.preprocessing import remove_stopwords

Remove stopwords and whitespaces

In [ ]:
def preprocess(row):
  text = row['Sentences']
  text = remove_stopwords(text)
  text = text.lower()
  text = text.replace('[^\w\s]',' ')
  text = text.replace('\s\s+', ' ')
  return text

In [ ]:
sexist_data['Sentences'] = sexist_data.apply(preprocess, axis=1)

In [ ]:
sexist_data

,Sentences,Label
0,"for woman, good",1
1,just dress sexy sign contract.,1
2,are having period ?,1
3,stop bitchy,1
4,i leave early children instead husband,1
...,...,...
1132,opportunities like night owls. they like strea...,0
1133,it's question love living. it's love love. the...,0
1134,a genius masters. an ordinary man tries master...,0
1135,"if want know real reasons attitude, think thou...",0


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
bow_vectorizer = CountVectorizer(max_df=0.90, min_df=2, max_features=1000, stop_words='english')
train_bow = bow_vectorizer.fit_transform(sexist_data['Sentences'])
train_bow.shape

(1137, 809)

In [ ]:
train_bow

<1137x809 sparse matrix of type '<class 'numpy.int64'>'
	with 4668 stored elements in Compressed Sparse Row format>

In [ ]:
test_bow = bow_vectorizer.transform(test_csv['tweet'])
test_bow.shape

(17197, 809)

In [ ]:
from sklearn.model_selection import train_test_split
xtrain_bow, xvalid_bow, ytrain, yvalid = train_test_split(train_bow, sexist_data['Label'],  
                                                          random_state=42, 
                                                          test_size=0.3)

In [ ]:
from sklearn import svm
svc = svm.SVC(kernel='linear', C=1, probability=True).fit(xtrain_bow, ytrain)

In [ ]:
y_pred_svm = svc.predict(xvalid_bow)

In [ ]:
test_svm = svc.predict(test_bow)

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
accuracy_score(yvalid, y_pred_svm) * 100

79.53216374269006

In [ ]:
import joblib

In [ ]:
# joblib.dump(bow_vectorizer, "/content/drive/MyDrive/twitter_sentiment/vectorizer.pkl")

In [ ]:
# joblib.dump(svc, "/content/drive/MyDrive/twitter_sentiment/anti_sexist_model.pkl")

In [ ]:
model = joblib.load("/content/drive/MyDrive/twitter_sentiment/anti_sexist_model.pkl")

In [ ]:
vectorizer = joblib.load("/content/drive/MyDrive/twitter_sentiment/vectorizer.pkl")

In [ ]:
model

SVC(C=1, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=True, random_state=None, shrinking=True, tol=0.001,
    verbose=False)

In [ ]:
vectorizer

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=0.9, max_features=1000, min_df=2,
                ngram_range=(1, 1), preprocessor=None, stop_words='english',
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [ ]:
from gensim.parsing.preprocessing import remove_stopwords
from sklearn.feature_extraction.text import CountVectorizer

def preprocess(text):
  """
  remove stopwords, whitespace, and make the sentence lowercase, tokenize
  """
  text = text.lower()
  text = text.replace('[^\w\s]',' ')
  text = text.replace('\s\s+', ' ')
  text = text.translate(str.maketrans('', '', string.punctuation))
  text = remove_stopwords(text)
  tokenized_text = text.split()
  print(text)
  text = vectorizer.transform(tokenized_text)
  
  return text


In [ ]:
a = preprocess("women are weak")

women weak


In [ ]:
b = preprocess("she's the best")

shes best


In [ ]:
model.predict(a)

array([1, 0])

In [ ]:
model.predict(b)

array([0, 0])